In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, AveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
source_training = pd.read_csv('metadata_processed.csv')
source_validation = pd.read_csv('icml_face_data.csv')

In [3]:
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
indexes = [x for x in range(7)]

In [4]:
try:
  for name in emotions:
    os.mkdir('Dataset/Training/'+name)
except FileExistsError as e:
  print('Error:', e)

Error: [WinError 183] Cannot create a file when that file already exists: 'Dataset/Training/Angry'


In [5]:
try:
  for name in emotions:
    os.mkdir('Dataset/Validation/'+name)
except FileExistsError as e:
  print('Error:', e)

Error: [WinError 183] Cannot create a file when that file already exists: 'Dataset/Validation/Angry'


In [6]:
for name in emotions:
  if len(os.listdir('Dataset/Training/{}'.format(name))) == 0:
    new_source = source_training[source_training['emotion'] == name]
    for i in range(new_source.shape[0]):
      img = new_source.iloc[i, 1]
      img = [x for x in img.split(' ')]
      img = np.array(img, dtype='float32')
      img = img.reshape(48,-1)
      cv2.imwrite('Dataset/Training/{}/{}.jpg'.format(name, i), img)
  else:
    print('{} was filled up with images!'.format(name))

Angry was filled up with images!
Disgust was filled up with images!
Fear was filled up with images!
Happy was filled up with images!
Sad was filled up with images!
Surprise was filled up with images!
Neutral was filled up with images!


In [7]:
for index, name in zip(indexes, emotions):
  if len(os.listdir('Dataset/Validation/{}'.format(name))) == 0:
    new_source = source_validation[source_validation['emotion'] == index]
    for i in range(500):
      img = new_source.iloc[i, 2]
      img = [x for x in img.split(' ')]
      img = np.array(img, dtype='float32')
      img = img.reshape(48,-1)
      cv2.imwrite('Dataset/Validation/{}/{}.jpg'.format(name, i), img)
  else:
    print('{} was filled up with images!'.format(name))

Angry was filled up with images!
Disgust was filled up with images!
Fear was filled up with images!
Happy was filled up with images!
Sad was filled up with images!
Surprise was filled up with images!
Neutral was filled up with images!


In [8]:
for name in emotions:
  print("Training", name, len(os.listdir(f'Dataset/Training/{name}')))
for name in emotions:
  print("Validation", name, len(os.listdir(f'Dataset/Validation/{name}')))

Training Angry 5126
Training Disgust 5126
Training Fear 5126
Training Happy 5126
Training Sad 5126
Training Surprise 5126
Training Neutral 5126
Validation Angry 500
Validation Disgust 500
Validation Fear 500
Validation Happy 500
Validation Sad 500
Validation Surprise 500
Validation Neutral 500


In [9]:
gen_train = ImageDataGenerator(rescale=1/255,
                               rotation_range=40, 
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True, 
                               fill_mode='nearest')

gen_valid = ImageDataGenerator(rescale=1/255)

img_train = gen_train.flow_from_directory('Dataset/Training/', target_size=(48, 48), batch_size=32, 
                                          class_mode='categorical', color_mode='grayscale')
img_valid = gen_valid.flow_from_directory('Dataset/validation/', target_size=(48, 48), batch_size=32, 
                                          class_mode='categorical', color_mode='grayscale')

Found 35882 images belonging to 7 classes.
Found 3500 images belonging to 7 classes.


In [10]:
#construct CNN structure
model = Sequential()

#1st convolution layer
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(7, activation='softmax'))

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(img_train, epochs=20, steps_per_epoch=256, validation_data=img_valid)

Epoch 1/20
256/256 [==============================] - 51s 198ms/step - loss: 1.9460 - accuracy: 0.1406 - val_loss: 1.9460 - val_accuracy: 0.1429
Epoch 2/20
256/256 [==============================] - 58s 228ms/step - loss: 1.9461 - accuracy: 0.1461 - val_loss: 1.9460 - val_accuracy: 0.1429
Epoch 3/20
256/256 [==============================] - 53s 208ms/step - loss: 1.9460 - accuracy: 0.1453 - val_loss: 1.9461 - val_accuracy: 0.1429
Epoch 4/20
 58/256 [=====>........................] - ETA: 33s - loss: 1.9463 - accuracy: 0.1401

In [ ]:
his = pd.DataFrame(history.history)
his.to_csv('bs64-e50.csv')